In [1]:
import bitstamp.client
import threading
import numpy as np

In [2]:
def print_costs(tickers, print_all=False):
    start_index = 0
    max_int_value = 100000.0

    usd_conversions = {x['base']:x for x in filter(lambda x : x['quote'] == 'USD', tickers)}
    usd_conversions['USD'] = {'bid':1.0, 'ask':1.0}
    unique_symbols = set(x['base'] for x in tickers) | set(x['quote'] for x in tickers)
    graph = np.array([[max_int_value]*len(unique_symbols)]*len(unique_symbols))


    #BTC is going to be the start node
    graph_symbol_lookup = {'BTC':0}

    for ticker in tickers:
        if ticker['base'] not in graph_symbol_lookup:
            graph_symbol_lookup[ticker['base']] = len(graph_symbol_lookup)
        if ticker['quote'] not in graph_symbol_lookup:
            graph_symbol_lookup[ticker['quote']] = len(graph_symbol_lookup)


        def get_cost(ticker, tickers):
            #Express cost in terms of USD since it is the only common denominator
            bid_conversion_rate = float(usd_conversions[ticker['quote']]['bid'])
            ask_conversion_rate = float(usd_conversions[ticker['quote']]['ask'])
            return float(ticker['ask'])*ask_conversion_rate-float(ticker['bid'])*bid_conversion_rate

        graph[graph_symbol_lookup[ticker['base']], graph_symbol_lookup[ticker['quote']]] = get_cost(ticker, tickers)
        graph[graph_symbol_lookup[ticker['quote']], graph_symbol_lookup[ticker['base']]] = get_cost(ticker, tickers)


    graph_symbol_reverse_lookup = {y:x for x, y in graph_symbol_lookup.items()}

    #Using a dijkstra like approch would be faster but Bitstamp has very few symbols so might as well start simple 
    for i in range(1, len(unique_symbols)):
        for j in range(1, len(unique_symbols)):
            
            if i == j:
                continue
            
            
            costs = []
            #From BTC to the first crypto (c1)
            costs.append(graph[graph_symbol_lookup['BTC']][i])
            #From the c1 to the second crypto(c2)
            costs.append(graph[i][j])
            #From c2 back to BTC
            costs.append(graph[j][graph_symbol_lookup['BTC']])

            if max_int_value in costs:
                continue

            cost = sum(costs)

            if print_all or cost <= 0:
                print('BTC -> {} -> {} -> BTC cost = {} USD'.format(graph_symbol_reverse_lookup[i], graph_symbol_reverse_lookup[j], cost))


In [ ]:
public_client = bitstamp.client.Public()
pairs = public_client.trading_pairs_info()


while True:
    try:
        tickers = []

        def get_ticker(base, quote):
            ticker = public_client.ticker(base=base, quote=quote)
            ticker['base'] = base
            ticker['quote'] = quote
            tickers.append(ticker)

        threads = []

        for pair in pairs:
            base, quote = pair['name'].split('/')
            thread = threading.Thread(target=get_ticker, args=(base, quote,))
            thread.start()
            threads.append(thread)

        for thread in threads:
            thread.join()

        print_costs(tickers, print_all=True)
    except:
        public_client = bitstamp.client.Public()

BTC -> ETH -> USD -> BTC cost = 12.694926365500805 USD
BTC -> ETH -> EUR -> BTC cost = 49.80555036550277 USD
BTC -> LTC -> USD -> BTC cost = 12.623491219700824 USD
BTC -> LTC -> EUR -> BTC cost = 48.566183719702735 USD
BTC -> XRP -> USD -> BTC cost = 11.8716180109008 USD
BTC -> XRP -> EUR -> BTC cost = 47.37475314750271 USD
BTC -> USD -> ETH -> BTC cost = 12.694926365500805 USD
BTC -> USD -> LTC -> BTC cost = 12.623491219700824 USD
BTC -> USD -> XRP -> BTC cost = 11.8716180109008 USD
BTC -> USD -> EUR -> BTC cost = 59.245312400003506 USD
BTC -> USD -> BCH -> BTC cost = 14.179984204900734 USD
BTC -> EUR -> ETH -> BTC cost = 49.80555036550277 USD
BTC -> EUR -> LTC -> BTC cost = 48.566183719702735 USD
BTC -> EUR -> XRP -> BTC cost = 47.37475314750271 USD
BTC -> EUR -> USD -> BTC cost = 59.245312400003506 USD
BTC -> EUR -> BCH -> BTC cost = 51.31810700490274 USD
BTC -> BCH -> USD -> BTC cost = 14.179984204900734 USD
BTC -> BCH -> EUR -> BTC cost = 51.31810700490274 USD
BTC -> ETH -> EUR ->

BTC -> BCH -> EUR -> BTC cost = 51.39705075710276 USD
BTC -> BCH -> USD -> BTC cost = 17.398927957099772 USD
BTC -> XRP -> EUR -> BTC cost = 47.37479986630271 USD
BTC -> XRP -> USD -> BTC cost = 14.991709672499782 USD
BTC -> EUR -> BCH -> BTC cost = 51.39705075710276 USD
BTC -> EUR -> XRP -> BTC cost = 47.3747998663027 USD
BTC -> EUR -> LTC -> BTC cost = 48.6638729923027 USD
BTC -> EUR -> USD -> BTC cost = 62.36531240000249 USD
BTC -> EUR -> ETH -> BTC cost = 49.945601382002735 USD
BTC -> LTC -> EUR -> BTC cost = 48.6638729923027 USD
BTC -> LTC -> USD -> BTC cost = 15.863651892299771 USD
BTC -> USD -> BCH -> BTC cost = 17.398927957099772 USD
BTC -> USD -> XRP -> BTC cost = 14.991709672499782 USD
BTC -> USD -> EUR -> BTC cost = 62.36531240000249 USD
BTC -> USD -> LTC -> BTC cost = 15.863651892299771 USD
BTC -> USD -> ETH -> BTC cost = 15.966213081999797 USD
BTC -> ETH -> EUR -> BTC cost = 49.945601382002735 USD
BTC -> ETH -> USD -> BTC cost = 15.966213081999797 USD
BTC -> LTC -> USD -> 

BTC -> ETH -> USD -> BTC cost = 3.551503271099989 USD
BTC -> ETH -> EUR -> BTC cost = 46.62126407110219 USD
BTC -> LTC -> USD -> BTC cost = 3.41747667270009 USD
BTC -> LTC -> EUR -> BTC cost = 45.638105372702185 USD
BTC -> USD -> ETH -> BTC cost = 3.551503271099989 USD
BTC -> USD -> LTC -> BTC cost = 3.41747667270009 USD
BTC -> USD -> XRP -> BTC cost = 2.671348389600073 USD
BTC -> USD -> EUR -> BTC cost = 47.16572000000223 USD
BTC -> USD -> BCH -> BTC cost = 4.689749526900073 USD
BTC -> XRP -> USD -> BTC cost = 2.6713483896000727 USD
BTC -> XRP -> EUR -> BTC cost = 44.49475629780216 USD
BTC -> EUR -> ETH -> BTC cost = 46.62126407110219 USD
BTC -> EUR -> LTC -> BTC cost = 45.638105372702185 USD
BTC -> EUR -> USD -> BTC cost = 47.16572000000223 USD
BTC -> EUR -> XRP -> BTC cost = 44.49475629780216 USD
BTC -> EUR -> BCH -> BTC cost = 48.12827992690222 USD
BTC -> BCH -> USD -> BTC cost = 4.689749526900073 USD
BTC -> BCH -> EUR -> BTC cost = 48.12827992690222 USD
BTC -> USD -> LTC -> BTC co

BTC -> XRP -> USD -> BTC cost = 16.4917538534016 USD
BTC -> XRP -> EUR -> BTC cost = 65.1497060328005 USD
BTC -> USD -> XRP -> BTC cost = 16.4917538534016 USD
BTC -> USD -> LTC -> BTC cost = 17.30622918230162 USD
BTC -> USD -> EUR -> BTC cost = 81.6400845000021 USD
BTC -> USD -> ETH -> BTC cost = 17.609719062101647 USD
BTC -> USD -> BCH -> BTC cost = 18.92094562730165 USD
BTC -> LTC -> USD -> BTC cost = 17.30622918230162 USD
BTC -> LTC -> EUR -> BTC cost = 66.33864568230054 USD
BTC -> EUR -> XRP -> BTC cost = 65.1497060328005 USD
BTC -> EUR -> USD -> BTC cost = 81.6400845000021 USD
BTC -> EUR -> LTC -> BTC cost = 66.33864568230054 USD
BTC -> EUR -> ETH -> BTC cost = 67.60384436210063 USD
BTC -> EUR -> BCH -> BTC cost = 69.19384052730061 USD
BTC -> ETH -> USD -> BTC cost = 17.609719062101647 USD
BTC -> ETH -> EUR -> BTC cost = 67.60384436210063 USD
BTC -> BCH -> USD -> BTC cost = 18.92094562730165 USD
BTC -> BCH -> EUR -> BTC cost = 69.19384052730061 USD
BTC -> BCH -> USD -> BTC cost = 

BTC -> ETH -> USD -> BTC cost = 18.024238546599918 USD
BTC -> ETH -> EUR -> BTC cost = 67.5258222465996 USD
BTC -> USD -> ETH -> BTC cost = 18.024238546599918 USD
BTC -> USD -> EUR -> BTC cost = 82.1162720999996 USD
BTC -> USD -> LTC -> BTC cost = 17.8003534754 USD
BTC -> USD -> XRP -> BTC cost = 17.0017684352 USD
BTC -> USD -> BCH -> BTC cost = 19.43063085120002 USD
BTC -> EUR -> ETH -> BTC cost = 67.5258222465996 USD
BTC -> EUR -> USD -> BTC cost = 82.1162720999996 USD
BTC -> EUR -> LTC -> BTC cost = 66.33139387539961 USD
BTC -> EUR -> XRP -> BTC cost = 65.1159306859996 USD
BTC -> EUR -> BCH -> BTC cost = 69.1484776511997 USD
BTC -> LTC -> USD -> BTC cost = 17.8003534754 USD
BTC -> LTC -> EUR -> BTC cost = 66.33139387539961 USD
BTC -> XRP -> USD -> BTC cost = 17.0017684352 USD
BTC -> XRP -> EUR -> BTC cost = 65.1159306859996 USD
BTC -> BCH -> USD -> BTC cost = 19.43063085120002 USD
BTC -> BCH -> EUR -> BTC cost = 69.1484776511997 USD
BTC -> ETH -> EUR -> BTC cost = 67.5258222465996 U

...

BTC -> XRP -> EUR -> BTC cost = 66.60826748050142 USD
BTC -> XRP -> USD -> BTC cost = 15.801715851199273 USD
BTC -> EUR -> XRP -> BTC cost = 66.60826748050141 USD
BTC -> EUR -> ETH -> BTC cost = 69.05370060160146 USD
BTC -> EUR -> BCH -> BTC cost = 70.36288397400142 USD
BTC -> EUR -> USD -> BTC cost = 82.4088153000007 USD
BTC -> EUR -> LTC -> BTC cost = 67.64434719960141 USD
BTC -> ETH -> EUR -> BTC cost = 69.05370060160146 USD
BTC -> ETH -> USD -> BTC cost = 16.902702901599298 USD
BTC -> BCH -> EUR -> BTC cost = 70.36288397400142 USD
BTC -> BCH -> USD -> BTC cost = 18.021143873999222 USD
BTC -> USD -> XRP -> BTC cost = 15.801715851199273 USD
BTC -> USD -> EUR -> BTC cost = 82.4088153000007 USD
BTC -> USD -> ETH -> BTC cost = 16.902702901599298 USD
BTC -> USD -> BCH -> BTC cost = 18.021143873999222 USD
BTC -> USD -> LTC -> BTC cost = 16.533506699599258 USD
BTC -> LTC -> EUR -> BTC cost = 67.64434719960141 USD
BTC -> LTC -> USD -> BTC cost = 16.533506699599258 USD
BTC -> USD -> EUR -> B

BTC -> BCH -> EUR -> BTC cost = 72.81930679230186 USD
BTC -> BCH -> USD -> BTC cost = 18.010497192299 USD
BTC -> EUR -> BCH -> BTC cost = 72.81930679230186 USD
BTC -> EUR -> LTC -> BTC cost = 70.03326932090181 USD
BTC -> EUR -> ETH -> BTC cost = 71.49975308640182 USD
BTC -> EUR -> XRP -> BTC cost = 69.05375890040182 USD
BTC -> EUR -> USD -> BTC cost = 84.84464910000088 USD
BTC -> LTC -> EUR -> BTC cost = 70.03326932090181 USD
BTC -> LTC -> USD -> BTC cost = 16.516595020899032 USD
BTC -> ETH -> EUR -> BTC cost = 71.49975308640182 USD
BTC -> ETH -> USD -> BTC cost = 16.89292158639904 USD
BTC -> XRP -> EUR -> BTC cost = 69.05375890040182 USD
BTC -> XRP -> USD -> BTC cost = 15.791373471099053 USD
BTC -> USD -> BCH -> BTC cost = 18.010497192299 USD
BTC -> USD -> EUR -> BTC cost = 84.84464910000088 USD
BTC -> USD -> LTC -> BTC cost = 16.516595020899032 USD
BTC -> USD -> ETH -> BTC cost = 16.89292158639904 USD
BTC -> USD -> XRP -> BTC cost = 15.791373471099053 USD
BTC -> XRP -> EUR -> BTC cos

BTC -> EUR -> USD -> BTC cost = 85.35647310000195 USD
BTC -> EUR -> LTC -> BTC cost = 69.60285965540223 USD
BTC -> EUR -> BCH -> BTC cost = 72.3319803894023 USD
BTC -> EUR -> XRP -> BTC cost = 68.53561299400224 USD
BTC -> EUR -> ETH -> BTC cost = 71.00591102030228 USD
BTC -> USD -> EUR -> BTC cost = 85.35647310000195 USD
BTC -> USD -> LTC -> BTC cost = 17.60181975539969 USD
BTC -> USD -> BCH -> BTC cost = 19.401346789399668 USD
BTC -> USD -> XRP -> BTC cost = 16.821403564699708 USD
BTC -> USD -> ETH -> BTC cost = 17.947255520299734 USD
BTC -> LTC -> EUR -> BTC cost = 69.60285965540223 USD
BTC -> LTC -> USD -> BTC cost = 17.60181975539969 USD
BTC -> BCH -> EUR -> BTC cost = 72.3319803894023 USD
BTC -> BCH -> USD -> BTC cost = 19.401346789399668 USD
BTC -> XRP -> EUR -> BTC cost = 68.53561299400224 USD
BTC -> XRP -> USD -> BTC cost = 16.821403564699708 USD
BTC -> ETH -> EUR -> BTC cost = 71.00591102030228 USD
BTC -> ETH -> USD -> BTC cost = 17.947255520299734 USD
BTC -> LTC -> USD -> BTC

BTC -> USD -> XRP -> BTC cost = 16.711400347200946 USD
BTC -> USD -> BCH -> BTC cost = 19.167649460600956 USD
BTC -> USD -> LTC -> BTC cost = 17.48048768320095 USD
BTC -> USD -> EUR -> BTC cost = 85.05451850000236 USD
BTC -> USD -> ETH -> BTC cost = 17.834650720301 USD
BTC -> XRP -> USD -> BTC cost = 16.711400347200946 USD
BTC -> XRP -> EUR -> BTC cost = 68.34365517650141 USD
BTC -> BCH -> USD -> BTC cost = 19.167649460600956 USD
BTC -> BCH -> EUR -> BTC cost = 72.13632846060153 USD
BTC -> LTC -> USD -> BTC cost = 17.48048768320095 USD
BTC -> LTC -> EUR -> BTC cost = 69.46561108320142 USD
BTC -> EUR -> USD -> BTC cost = 85.05451850000236 USD
BTC -> EUR -> XRP -> BTC cost = 68.3436551765014 USD
BTC -> EUR -> BCH -> BTC cost = 72.13632846060153 USD
BTC -> EUR -> LTC -> BTC cost = 69.46561108320142 USD
BTC -> EUR -> ETH -> BTC cost = 70.81135162030148 USD
BTC -> ETH -> USD -> BTC cost = 17.834650720301 USD
BTC -> ETH -> EUR -> BTC cost = 70.81135162030148 USD
BTC -> LTC -> EUR -> BTC cost

BTC -> BCH -> USD -> BTC cost = 19.797096231500348 USD
BTC -> BCH -> EUR -> BTC cost = 71.67172753150169 USD
BTC -> USD -> BCH -> BTC cost = 19.797096231500348 USD
BTC -> USD -> XRP -> BTC cost = 17.351289044700366 USD
BTC -> USD -> EUR -> BTC cost = 85.21047080000199 USD
BTC -> USD -> ETH -> BTC cost = 18.48976844280037 USD
BTC -> USD -> LTC -> BTC cost = 18.146225210300358 USD
BTC -> XRP -> USD -> BTC cost = 17.351289044700366 USD
BTC -> XRP -> EUR -> BTC cost = 67.85962617400162 USD
BTC -> EUR -> BCH -> BTC cost = 71.67172753150169 USD
BTC -> EUR -> USD -> BTC cost = 85.21047080000199 USD
BTC -> EUR -> XRP -> BTC cost = 67.85962617400162 USD
BTC -> EUR -> ETH -> BTC cost = 70.34242164280164 USD
BTC -> EUR -> LTC -> BTC cost = 69.00991271030163 USD
BTC -> ETH -> USD -> BTC cost = 18.48976844280037 USD
BTC -> ETH -> EUR -> BTC cost = 70.34242164280164 USD
BTC -> LTC -> USD -> BTC cost = 18.146225210300358 USD
BTC -> LTC -> EUR -> BTC cost = 69.00991271030163 USD
BTC -> XRP -> USD -> B